In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Varshini C\\Desktop\\chatgen\\petcare\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Varshini C\\Desktop\\chatgen\\petcare'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [17]:
#extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1456


In [ ]:
 #text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\Varshini C\AppData\Local\Temp\ipykernel_20512\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Varshini C\Desktop\chatgen\chatgenn\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [23]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [24]:
query_result

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [27]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "petcare"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [15]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [20]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

NameError: name 'embeddings' is not defined

In [30]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [31]:
docsearch

In [32]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [33]:
retrieved_docs = retriever.invoke("How to train dogs?")

In [34]:
retrieved_docs

[Document(id='4dad5a2d-9d00-446b-a911-6ef94eb1ebec', metadata={'page': 8.0, 'page_label': '9', 'source': 'Data\\03. Training Rex in the City - Basic Obedience for Dogs author Nancy Clarke.pdf'}, page_content='and practise with the dog. Remember \nto be patient.\n 8. Practise. Practise. Practise. Dogs learn \nby repetition. Set aside time each day \nto train your dog. At the beginner level, \na minimum of 30 minutes of practice \nper day is usually needed. It’s always \nbetter to train for shorter periods, a \ncouple of times a day, especially with \nyounger or inexperienced dogs.\n 9. Rough or abusive treatment will not \nserve your training well. If a dog \ndisobeys your commands, there is a'),
 Document(id='a07fb505-6c7e-4d63-b3a4-cb8215f13d08', metadata={'page': 0.0, 'page_label': '1', 'source': 'Data\\03. Training Rex in the City - Basic Obedience for Dogs author Nancy Clarke.pdf'}, page_content='BASIC OBEDIENCE \nFOR DOGS\nTRAINING'),
 Document(id='2c4c573a-3501-44ef-8fb7-80442426

In [35]:

from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.4,
    max_tokens=500
)


In [36]:
system_prompt = (
    "You are an AI assistant for a pet care and training chatbot. "
    "Use the provided context to answer user queries accurately. "
    "If the answer is unknown, state that you don't know. "
    "Ensure responses are clear, structured, and helpful."
    "\n\n"
    "Response Format:\n"
    "- Provide answers in points with subheadings.\n"
    "- Keep explanations concise but informative.\n"
    "- Offer actionable advice for pet care and training."
    "\n\n"
    "{context}"
)


In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)


In [39]:
from langchain.chains import create_retrieval_chain

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input": "What is LABRADOR?"})
print(response["answer"])


Labrador Retriever:
- Origin: Newfoundland, Canada.
- Breed history: Initially used for fishing purposes, later crossed with Setters, Spaniels, and other retrievers.
- Temperament: Cheerful, good with children and other pets.
- Appearance: Webbed paws, athletic, available in three colors (yellow, black, chocolate), and weighing between 55-75 pounds.
- Suitable dog door: PlexiDor large.

Landseer:
- Origin: Canada, named after British painter Sir Edwin Henry Landseer.
- Breed history: Known for the painting "A Distinguished Member of the Humane Society" featuring the breed.

Additional information:
- The Labrador Retriever is the most popular dog breed in Canada, the United States, and the United Kingdom.
- Often chosen as assistance dogs, including guide dogs for the blind and police dogs.

If you have any questions about these breeds or need advice on pet care and training, feel free to ask!
